## Preprocess the Sim2Real dataset

The Sim2Real dataset uses jpeg images to represent wildfire masks. This representation is very lightweight but slow to work with. 
To benchmark faster, we convert them to binary NumPy files. These files are ~100 times heavier and it requires an overhead time to convert from jpeg to them, but once they are created and stored, they allow 50--100x faster operations.

1. Download the dataset from [Sim2Real-Fire GitHub repository](https://github.com/TJU-IDVLab/Sim2Real-Fire).
2. Extract all files from the dataset into your `Dataset` folder
3. Run the preprocessing function to convert from the jpeg representation to the NumPy one, and compute burn maps

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
# import requred modules
import sys
import os
import time
import numpy as np

# Add code to path
module_path = os.path.abspath(".") + "/code"
if module_path not in sys.path:
    sys.path.append(module_path)

from dataset import preprocess_sim2real_dataset, load_scenario_npy, compute_and_save_burn_maps_sim2real_dataset
from Strategy import RandomDroneRoutingStrategy, return_no_custom_parameters, SensorPlacementOptimization, RandomSensorPlacementStrategy, DroneRoutingOptimizationExample, MY_DRONE_STRATEGY, MY_SENSOR_STRATEGY, LoggedOptimizationSensorPlacementStrategy,DroneRoutingOptimizationSlow
from benchmark import run_benchmark_scenario,run_benchmark_scenarii_sequential, benchmark_on_sim2real_dataset, get_burnmap_parameters
from displays import create_scenario_video

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Initializing the Julia session. This can take up to 1 minute.
initializing the ground sensor julia module
installing packages


   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/julia_env/Manifest.toml`
   Resolving package versions...
  No Changes to `/opt/anaconda3/julia_env/Project.toml`
  No Changes to `/opt/anaconda3/jul

initializing the drone julia module
Julia session initialized.


Uncomment the cell below to preprocess the dataset.
- `n_max_scenarii_per_layout` controls the number of scenarios we convert from jpeg to NumPy files for each layout.
- If executed on all ~50 layouts of the dataset, the code below takes ~15mins to run and generates **400 GB** of data. Make sure to have the space.

In [2]:
# preprocess_sim2real_dataset("WideDataset/", n_max_scenarii_per_layout=100) 

We can now run the benchmark function on any scenario.
1. We load the scenario using `load_scenario_npy` since it is in preprocessed `npy` format
2. Werun the benchmark using `run_benchmark_scenario` that takes as input:
    - The `scenario`, 
    - The sensor placement strategy and the drone routing strategy
    - A dictionary `custom_initialization_parameters` that contains any custom initialization inputs for your strategy functions (such as the burn map if your strategy needs a burn map as input)
    - A python function `custom_step_parameters_function` that returns a dictionary of custom inputs for your routing function. This function is executed by the benchmarking code at each time step. Your routing function will be called with: `routing(custom_step_parameters_function())` internally

In [12]:
# change values here to change benchmarking parameters
def my_automatic_layout_parameters(scenario:np.ndarray):
    return {
        "N": scenario.shape[1],
        "M": scenario.shape[2],
        "max_battery_distance": 4,
        "max_battery_time": 5,
        "n_drones": 3,
        "n_ground_stations": 4,
        "n_charging_stations": 4,
    }

In [7]:
params = my_automatic_layout_parameters(scenario) 
params['charging_stations_locations'] = [(1,1)]
params['ground_sensor_locations'] = [(1,1)]
DroneRoutingOptimizationSlow(params, {"burnmap_filename": "./WideDataset/0001/burn_map.npy", "load_from_logfile": True, "reevaluation_step": 3, "optimization_horizon": 7}).get_initial_drone_locations()

calling julia optimization model
--- parameters for julia ---
burnmap_filename: ./WideDataset/0001/burn_map.npy
n_drones: 1
charging_stations_locations: [(1, 1)]
ground_sensor_locations: [(1, 1)]
optimization_horizon: 10
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Creating model took 0.043317792005836964 seconds
Optimizing model took 0.003708832897245884 seconds
Solver Status: OPTIMAL
Objective Value: 0.0010000000000000002
Took 0.04776045889593661 seconds
initial optimization finished


[(2, 2)]

In [13]:
# That's very fast to run!
print("starting benchmark")
time_start = time.time()
scenario = load_scenario_npy("WideDataset/0001/scenarii/0001_00002.npy")
print("scenario loaded")
device, delta_t, _ = run_benchmark_scenario(scenario, RandomSensorPlacementStrategy, DroneRoutingOptimizationSlow, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy", "load_from_logfile": True, "reevaluation_step": 2, "optimization_horizon":8}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


starting benchmark
scenario loaded
calling julia optimization model
--- parameters for julia ---
burnmap_filename: ./WideDataset/0001/burn_map.npy
n_drones: 1
charging_stations_locations: [(56, 71), (46, 92), (9, 24), (97, 114)]
ground_sensor_locations: [(4, 5), (47, 78), (85, 63), (40, 76)]
optimization_horizon: 8
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Creating model took 25.157615167088807 seconds
Optimizing model took 0.11378416698426008 seconds
Solver Status: OPTIMAL
Objective Value: 0.06112501854804485
Took 25.27240899996832 seconds
initial_positions_only: False
current_solution: [[("fly", (98, 115))], [("fly", (99, 115))]]
initial optimization finished
calling julia optimization model
--- parameters for julia ---
charging_stations_locations: [(57, 72), (47, 93), (10, 25), (98, 115)]
ground_sensor_locations: [(5, 6), (48, 79), (86, 64), (41, 77)]
drone_locations: [(99, 115)]
drone_states: 

JuliaError: Result index of attribute MathOptInterface.VariablePrimal(1) out of bounds. There are currently 0 solution(s) in the model.
Stacktrace:
  [1] check_result_index_bounds
    @ ~/.julia/packages/MathOptInterface/92oIV/src/attributes.jl:207 [inlined]
  [2] get(model::Gurobi.Optimizer, attr::MathOptInterface.VariablePrimal, x::MathOptInterface.VariableIndex)
    @ Gurobi ~/.julia/packages/Gurobi/yFYMV/src/MOI_wrapper/MOI_wrapper.jl:3059
  [3] get(b::MathOptInterface.Bridges.LazyBridgeOptimizer{Gurobi.Optimizer}, attr::MathOptInterface.VariablePrimal, index::MathOptInterface.VariableIndex)
    @ MathOptInterface.Bridges ~/.julia/packages/MathOptInterface/92oIV/src/Bridges/bridge_optimizer.jl:1252
  [4] get(model::MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.Bridges.LazyBridgeOptimizer{Gurobi.Optimizer}, MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Utilities.Model{Float64}}}, attr::MathOptInterface.VariablePrimal, index::MathOptInterface.VariableIn

Objective Value: No solution found


We can visualize the strategy in action by creating a video of the scenario. 
1. Use the `return_history` parameter or `run_benchmark_scenario` to output the log of ground sensor and drone positions during the benchmark
2. Use the `create_scenario_video` function to compile the video. This can take a couple seconds

In [4]:
delta , device , (position_history, ground, charging)  = run_benchmark_scenario(scenario, SensorPlacementOptimization, RandomDroneRoutingStrategy, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy"}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
create_scenario_video(scenario[:len(position_history)],drone_locations_history=position_history,starting_time=0,out_filename='test_simulation', ground_sensor_locations = ground, charging_stations_locations = charging)

calling julia optimization model
In julia fuinction
Set parameter Username
Set parameter LicenseID to value 2612529
Academic license - for non-commercial use only - expires 2026-01-21
Took 8.259267374873161 seconds
optimization finished
Video saved at: display_test_simulation/test_simulation.mp4


Instead of running a benchmark on a single scenario, we are interested in running the benchmark on all scenarii of a given layout (potentially in parallel!)

1. We use `run_benchmark_scenarii_sequential`

In [6]:
run_benchmark_scenarii_sequential("WideDataset/0001/scenarii/", SensorPlacementOptimization, RandomDroneRoutingStrategy, custom_initialization_parameters_function = get_burnmap_parameters, custom_step_parameters_function = return_no_custom_parameters, starting_time=0, max_n_scenarii=100)

  0%|          | 0/100 [00:00<?, ?it/s]

calling julia optimization model
In julia fuinction
Loading burn map from WideDataset/0001/scenarii/burn_map.npy


JuliaError: Error loading burn map: SystemError("opening file \"WideDataset/0001/scenarii/burn_map.npy\"", 2, nothing)
Stacktrace:
 [1] error(s::String)
   @ Base ./error.jl:35
 [2] load_burn_map(filename::String)
   @ Main ~/Desktop/Climate/wildfire_drone_routing/julia/helper_functions.jl:10
 [3] ground_charging_opt_model_grid(risk_pertime_file::String, N_grounds::Int64, N_charging::Int64)
   @ Main ~/Desktop/Climate/wildfire_drone_routing/julia/ground_charging_opt.jl:88
 [4] pyjlany_call(self::typeof(ground_charging_opt_model_grid), args_::Py, kwargs_::Py)
   @ PythonCall.JlWrap ~/.julia/packages/PythonCall/WMWY0/src/JlWrap/any.jl:47
 [5] _pyjl_callmethod(f::Any, self_::Ptr{PythonCall.C.PyObject}, args_::Ptr{PythonCall.C.PyObject}, nargs::Int64)
   @ PythonCall.JlWrap ~/.julia/packages/PythonCall/WMWY0/src/JlWrap/base.jl:73
 [6] _pyjl_callmethod(o::Ptr{PythonCall.C.PyObject}, args::Ptr{PythonCall.C.PyObject})
   @ PythonCall.JlWrap.Cjl ~/.julia/packages/PythonCall/WMWY0/src/JlWrap/C.

On all of the dataset!

In [ ]:
benchmark_on_sim2real_dataset("WideDataset/", SensorPlacementStrategy, DroneRoutingStrategy, custom_initialization_parameters_function = lambda x: None, custom_step_parameters_function = return_no_custom_parameters, max_n_scenarii=100, starting_time=0)

In [4]:
# That's very fast to run!
time_start = time.time()
scenario = load_scenario_npy("WideDataset/0001/scenarii/0001_00002.npy")
device, delta_t, _ = run_benchmark_scenario(scenario, RandomSensorPlacementStrategy, MY_DRONE_STRATEGY, custom_initialization_parameters = {"burnmap_filename": "./WideDataset/0001/burn_map.npy", "call_every_n_steps": 5, "optimization_horizon": 10}, custom_step_parameters_function = return_no_custom_parameters, automatic_initialization_parameters_function=my_automatic_layout_parameters, return_history=True)
print("Fire detected in ", delta_t, "time steps by device: ", device)
print(f"Time taken to run benchmark on the scenario: {time.time() - time_start} seconds")


calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling julia optimization model
optimization finished
calling ju

In [13]:
# define function to get parameters for optimal drone routing strategy
# def get_routing_optimization_parameters(input_dir:str):
#     return {
#         "burnmap_filename": f"{input_dir}/burn_map.npy",
#         "call_every_n_steps": 5,
#         "optimization_horizon": 10
#     }

# add the log
